In [1]:
import numpy as np
import pandas as pd
import p2pspatial

import sklearn.base as sklb
import sklearn.model_selection as sklms

2018-01-06 17:58:32,030 [pulse2percept] [INFO] Welcome to pulse2percept


In [2]:
class DummyRegressor(sklb.BaseEstimator, sklb.RegressorMixin):
    def __init__(self, a=1):
        self.a = a
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def predict(self, X):
        y_pred = []
        for i, _ in X.iterrows():
            y_pred.append(i * self.a)
        return np.array(y_pred)

In [3]:
dummy = DummyRegressor(a=0)

In [4]:
search_params = {'a': (-100, 100)}
pso = p2pspatial.model_selection.ParticleSwarmOptimizer(dummy, search_params, verbose=False)
print(pso)

ParticleSwarmOptimizer(estimator=DummyRegressor(a=0), max_iter=100,
            min_func=0.0001, search_params={'a': (-100, 100)},
            swarm_size=10, verbose=False)


In [5]:
n_samples = 12
X = pd.DataFrame(np.repeat(np.arange(n_samples), 2).reshape((-1, 2)), columns=['feat1', 'feat2'])
y = pd.DataFrame(np.arange(n_samples), columns=['target'])

In [6]:
pso.fit(X, y=y)

Stopping search: Swarm best objective change less than 0.0001
Best err: -0.999982292198 Best params: {'a': 0.99776295311952801}


In [7]:
pso.score(X, y)

0.99998229219828527

In [8]:
y_test, y_pred, best_params = p2pspatial.model_selection.crossval_predict(pso, X, y, n_folds=5)

Stopping search: Swarm best objective change less than 0.0001
Best err: -0.999984528639 Best params: {'a': 1.0013611971275169}
Stopping search: Swarm best objective change less than 0.0001
Best err: -0.99999991705 Best params: {'a': 1.0001549579701183}
Stopping search: Swarm best objective change less than 0.0001
Best err: -0.999984653858 Best params: {'a': 0.9977401603599434}
Stopping search: Swarm best objective change less than 0.0001
Best err: -0.999999996946 Best params: {'a': 1.0000319794217996}
Stopping search: Swarm best objective change less than 0.0001
Best err: -0.999980871767 Best params: {'a': 1.0023531086160005}


In [9]:
p2pspatial.model_selection.crossval_score(y_test, y_pred, metric='r2')

[0.99999536785595011,
 0.99999939970068741,
 0.99913183121620885,
 0.99999970342180855,
 0.99755259288985654]